In [2]:
import pandas as pd
from pathlib import Path

In [3]:
here = Path("__file__").parent
data_folder = here / "data"

roster = pd.read_csv(
    data_folder/"roster.csv",
    converters = {
                "NetID": str.lower, 
                "Email Address": str.lower
                },
    usecols = ["Section", "Email Address", "NetID"],
    index_col = "NetID"
)

# dataFrames.
hwcsv = pd.read_csv(
    data_folder/"hw_exam_grades.csv",
    converters = {
                "Email Address": str.lower,
                "SID": str.lower,
                "First Name": str.lower,
                "Last Name": str.lower
    },
    usecols= lambda x: "Submission" not in x,
    index_col = "SID"
)

quiz_grades = pd.DataFrame()

for file_path in data_folder.glob("quiz_*_grades.csv"):
    quiz_name = " ".join(file_path.stem.title().split("_")[:2])
    # print(file_path)
    quiz = pd.read_csv(
        file_path,
        converters = {
            "Email": str.lower
        },
        usecols=["Email", "Grade"],
        index_col="Email"
    ).rename(columns={"Grade": quiz_name})
    quiz_grades = pd.concat([quiz_grades, quiz], axis=1)



In [4]:
finalData = pd.merge(
    roster,
    hwcsv,
    left_index=True,
    right_index=True
)

finalData = pd.merge(
    finalData,
    quiz_grades,
    left_on = "Email Address",
    right_index = True
    
)

finalData = finalData.fillna(0)

# Average Exam score

for i in range(1,4):
    finalData[f"Exam {i} Score"] = (finalData[f"Exam {i}"] / finalData[f"Exam {i} - Max Points"])

In [8]:
    
homework_score = finalData.filter(regex=r"^Homework \d\d?$", axis = 1)
homework_max = finalData.filter(regex=r"^Homework \d\d? -", axis=1)

sum_hw_scores = homework_score.sum(axis=1)
sum_hw_maxpoints = homework_max.sum(axis=1)

finalData["Total Homework"] = sum_hw_scores / sum_hw_maxpoints

hw_max_renamed = homework_max.set_axis(homework_score.columns, axis=1)
hwaverage = ((homework_score / hw_max_renamed).round(2)).sum(axis=1)

quiz = finalData.filter(regex=r"^Quiz \d$", axis=1)
quiz_sum = quiz.sum(axis=1)

quiz_max_points = pd.Series(
    {"Quiz 1": 11, "Quiz 2": 15, "Quiz 3": 17, "Quiz 4": 14, "Quiz 5": 12}
)
quizmaxsum = quiz_max_points.sum()

quizaverage = list()

for i in quiz_sum.index:
    quizaverage.append(((quiz_sum[i]/quizmaxsum)*10).round(2))

quizavg = pd.DataFrame()





In [9]:
weightage = pd.Series({
    "Exam 1 score": 0.15,
    "Exam 2 score": 0.15,
    "Exam 3 score": 0.1,
    "Quiz Score" : 0.30,
    "Homework Score": 0.30    
})



# finalData
quizaverageactual

Data    [6.09, 6.81, 5.94, 4.93, 6.52, 8.26, 5.51, 6.3...
dtype: object